In [9]:
import numpy as np
import xgboost as xgb
import pandas as pd
# import category_encoders as ce
import sklearn
# from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
# import csv


In [10]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = test_data['id']

train_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)

labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
# Assigning numerical values and storing in another column
train_data['sub_grade'] = labelencoder.fit_transform(train_data['sub_grade'])
train_data['home_ownership'] = labelencoder.fit_transform(train_data['home_ownership'])
train_data['emp_length'].replace('< 1 year', 0.5, inplace=True)
train_data['emp_length'].replace('1 year', 1.0, inplace=True)
train_data['emp_length'].replace('2 years', 2.0, inplace=True)
train_data['emp_length'].replace('3 years', 3.0, inplace=True)
train_data['emp_length'].replace('4 years', 4.0, inplace=True)
train_data['emp_length'].replace('5 years', 5.0, inplace=True)
train_data['emp_length'].replace('6 years', 6.0, inplace=True)
train_data['emp_length'].replace('7 years', 7.0, inplace=True)
train_data['emp_length'].replace('8 years', 8.0, inplace=True)
train_data['emp_length'].replace('9 years', 9.0, inplace=True)
train_data['emp_length'].replace('10 years', 10.0, inplace=True)
train_data['emp_length'].replace('10+ years', 15.0, inplace=True)
train_data['emp_length'] = train_data['emp_length'].fillna(0)

test_data['sub_grade'] = labelencoder2.fit_transform(test_data['sub_grade'])
test_data['home_ownership'] = labelencoder2.fit_transform(test_data['home_ownership'])
test_data['emp_length'].replace('< 1 year', 0.5, inplace=True)
test_data['emp_length'].replace('1 year', 1.0, inplace=True)
test_data['emp_length'].replace('2 years', 2.0, inplace=True)
test_data['emp_length'].replace('3 years', 3.0, inplace=True)
test_data['emp_length'].replace('4 years', 4.0, inplace=True)
test_data['emp_length'].replace('5 years', 5.0, inplace=True)
test_data['emp_length'].replace('6 years', 6.0, inplace=True)
test_data['emp_length'].replace('7 years', 7.0, inplace=True)
test_data['emp_length'].replace('8 years', 8.0, inplace=True)
test_data['emp_length'].replace('9 years', 9.0, inplace=True)
test_data['emp_length'].replace('10 years', 10.0, inplace=True)
test_data['emp_length'].replace('10+ years', 15.0, inplace=True)
test_data['emp_length'] = test_data['emp_length'].fillna(0)

train_data['mort_acc'] = train_data['mort_acc'].fillna(0)

test_data['mort_acc'] = test_data['mort_acc'].fillna(0)
# Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]

X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_test.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0.0, inplace=True)
y_train_numeric.replace('Charged Off', 1.0, inplace=True)


## Adding artificial Data

In [11]:
print("BEFORE")
print(X_train_numeric.shape)
print(y_train_numeric.shape)

X_balanced = []
Y_balanced = []
for i in range(X_train_numeric.shape[0]):
    curr_df = X_train_numeric.iloc[i,:]
    d = curr_df.to_dict()
    label = y_train_numeric[i]
    if label == 1:
        for _ in range(3):
            X_balanced.append(d)
            Y_balanced.append(label)
    X_balanced.append(d)
    Y_balanced.append(label)
X_train_numeric = pd.DataFrame(X_balanced)
y_train_numeric= pd.Series(Y_balanced)

print('AFTER')
print(X_train_numeric.shape)
print(y_train_numeric.shape)

BEFORE
(197250, 16)
(197250,)
AFTER
(287922, 16)
(287922,)


In [12]:
X_train_numeric.head()

,loan_amnt,term_(months),int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
0,5000.0,36.0,10.65,162.87,6.0,15.0,4.0,24000.0,27.65,3.0,0.0,13648.0,83.7,9.0,0.0,0.0
1,2500.0,60.0,15.27,59.83,13.0,0.5,4.0,30000.0,1.00,3.0,0.0,1687.0,9.4,4.0,0.0,0.0
2,2500.0,60.0,15.27,59.83,13.0,0.5,4.0,30000.0,1.00,3.0,0.0,1687.0,9.4,4.0,0.0,0.0
3,2500.0,60.0,15.27,59.83,13.0,0.5,4.0,30000.0,1.00,3.0,0.0,1687.0,9.4,4.0,0.0,0.0
4,2500.0,60.0,15.27,59.83,13.0,0.5,4.0,30000.0,1.00,3.0,0.0,1687.0,9.4,4.0,0.0,0.0


In [13]:
# xg_train, xg_test, xg_ytrain, xg_ytest = sklearn.model_selection.train_test_split(
#     X_train_numeric, y_train_numeric, test_size=0.2, random_state=0)

# print(xg_train.shape, xg_test.shape, xg_ytrain.shape, xg_ytest.shape)
# print(type(xg_train), type(xg_test), type(xg_ytrain), type(xg_ytest))

In [15]:
# def auc(m, xtrain, xtest): 
#     return (metrics.roc_auc_score(y_train_numeric,m.predict_proba(xtrain)[:,1]),
#                             metrics.roc_auc_score(xg_ytest,m.predict_proba(xtest)[:,1]))

import warnings
warnings.filterwarnings("ignore")

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10],
              "min_child_weight" : [2],
              "n_estimators": [200],
              "learning_rate": [0.01],
              "gamma" : [0]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(X_train_numeric, y_train_numeric)

print(grid_search.best_estimator_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3; 1/1] START gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=2, n_estimators=200
[CV 3/3; 1/1] START gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=2, n_estimators=200
[CV 1/3; 1/1] START gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=2, n_estimators=200


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

[16:58:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:58:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:58:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/3; 1/1] END gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=2, n_estimators=200;, score=0.629 total time= 1.5min
[CV 2/3; 1/1] END

KeyboardInterrupt: 

In [17]:
model = xgb.XGBClassifier(max_depth=10, min_child_weight=2,  n_estimators=100,
                          n_jobs=-1, verbose=1,learning_rate=0.01, gamma=4, random_state=0)
model.fit(X_train_numeric, y_train_numeric)
predictions1 = model.predict_proba(X_test_numeric)[:,1]
# predictions2 = model.predict_proba(xg_test)
# print(predictions2)

# print(auc(model, xg_train, xg_test))

predictions = {"id": [], "loan_status": []}
predictions["id"] = id_column
predictions["loan_status"] = (predictions1)


[17:00:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:00:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [18]:
predictions_df = pd.DataFrame(data=predictions)
predictions_df.head()

,id,loan_status
0,200000,0.224322
1,200001,0.449637
2,200002,0.271287
3,200003,0.397399
4,200004,0.387230


In [19]:
predictions_df.to_csv("xgb_predictions", index=False)